In [ ]:
"""
Purpose: To develop the full method for 
creating the proofread neuron and saving it off in the database

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

INFO - 2021-02-04 07:56:39,899 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-04 07:56:39,901 - settings - Setting database.user to celiib
INFO - 2021-02-04 07:56:39,901 - settings - Setting database.password to newceliipass
INFO - 2021-02-04 07:56:39,912 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-04 07:56:39,913 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-04 07:56:39,926 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-04 07:56:40,205 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-04 07:56:42,676 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-04 07:56:42,957 - settings - Setting enable_python_native_blobs to True


# Importing the Neuron

In [5]:
minnie.Decomposition() & "n_somas > 1"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134765143804,0,0.25,30.00,0,0,1,=BLOB=,44570,90882,0,1,1,1,2,1,1,1,28585.105732166514,28585.105732166514,28585.106,1726.047,1726.047,1726.047,1726.047,4,0.00013993301397863443,4.0,28585.106,1,0.00013993301266750595,4.0,174766472.25666714,13946453.78167041,6113.899801322207,6113.899744036882,174766472.25666714,89.0604
864691134768966140,0,0.25,30.00,5,0,1,=BLOB=,81637,164711,0,1,1,1,2,2,5,4,61206.675450776704,40401.85629631666,4713.508,108.165,108.165,1174.845,1135.231,2,3.2676174375921285e-05,0.4,6413.497,2,0.0003118423537112436,1.0,806371663.6001841,403185831.80009204,13174.570545800676,125730.41877156628,403185831.80009204,58.3114
864691134831878402,0,0.25,30.00,0,0,1,=BLOB=,141431,288006,0,1,0,1,2,3,5,3,167837.6994974118,105050.86635215634,42893.926,114.293,114.293,1035.029,1035.029,0,0.0,0.0,19892.907,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.6019
864691134874936141,0,0.25,30.00,0,0,1,=BLOB=,64451,129722,0,2,2,1,2,3,7,3,69751.43119438276,56515.80323426945,5610.567,124.571,124.571,1568.841,1568.841,4,5.7346493563018516e-05,0.5714285714285714,41954.196,3,9.534207257839e-05,1.333,123117238.4626452,18270225.275904972,1765.0854807486746,2934.5631712891172,41039079.4875484,143.5662
864691134884769018,0,0.25,30.00,0,0,1,=BLOB=,1528011,3078802,0,7,5,2,3,20,421,129,15358033.818095537,6359831.284514944,19558.966,117.674,117.674,462.331,428.373,3853,0.0002508784682750353,9.152019002375297,8180017.673,186,0.00047102587720778387,20.715,818307027581.6089,118226275.54461034,53282.01755991983,100037.31780221165,4399500148.288219,8443.518
864691134884772346,0,0.25,30.00,0,0,1,=BLOB=,385614,805879,0,1,1,1,2,10,106,51,3416069.9194424567,1568051.5996385978,22918.888,186.605,186.605,506.188,456.498,1063,0.0003111763005639809,10.028301886792454,2126475.354,56,0.0004998882296004264,18.982,207947661950.84503,109383813.78330484,60873.36233000306,97789.82933410713,37133511

In [19]:
segment_id = 864691134884796410
verbose = True

# Step 1: Pulling Down the  Neurons

In [8]:
neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 3


# Getting the Nuclei That Correspond

In [66]:
nucleus_ids,nucleus_centers = du.segment_to_nuclei(segment_id)

if verbose:
    print(f"Number of Corresponding Nuclei = {len(nucleus_ids)}")
    print(f"nucleus_ids = {nucleus_ids}")
    print(f"nucleus_centers = {nucleus_centers}")

Number of Corresponding Nuclei = 3
nucleus_ids = [112158 112369 159055]
nucleus_centers = [[450112 580480 594040]
 [433664 574784 677960]
 [473856 673728 727880]]


In [135]:
from pykdtree.kdtree import KDTree

original_mesh = du.fetch_segment_id_mesh(filtered_neuron.segment_id)
original_mesh_kdtree = KDTree(original_mesh.triangles_center)

# For Each Neuron

In [13]:
curr_idx = 1
neuron_obj = neuron_objs[curr_idx]
split_index = neuron_split_idxs[curr_idx]

In [18]:
nviz.visualize_neuron(neuron_obj,
                      limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Part A: Proofreading the Neuron

In [119]:
filtering_info

'filtering_info'

In [121]:
import proofreading_utils as pru
output_dict= pru.proofread_neuron(neuron_obj,
                    plot_limb_branch_filter_with_disconnect_effect=False,
                    plot_final_filtered_neuron=False,
                    verbose=False)

filtered_neuron = output_dict["filtered_neuron"]
split_info = output_dict["split_info"]
cell_type_info = output_dict["cell_type_info"]
filtering_info = output_dict["filtering_info"]

---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 26.615037202835083


 ------ Part C: Inhibitory Excitatory Classification ---- 


Did not pass final filters to continuing
Did not pass final filters to continuing
Did not pass final filters to continuing
Inhibitory Excitatory Classification = 10.65115475654602
cell type = excitatory


 ------ Part D: Neuron Filtering ---- 

 

--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': True}



--- Working on filter 1:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 2:
function = dendrite_on_axon_merges
function __name__ = filter_away_dendrite_on_axon_merges
function arguments = {'use_pre_existing_axon_labels': True}



--- Working on filte

# Part B: Getting Soma Centers and Matching To Nuclei

In [139]:
winning_nucleus_id, nucleus_info = nru.pair_segment_id_to_nuclei(neuron_obj,
                             "S0",
                              nucleus_ids,
                              nucleus_centers,
                             nuclei_distance_threshold = 20000,
                              return_matching_info = True,
                             verbose=True)

print(f"winning_nucleus_id = {winning_nucleus_id}")

soma_center = [473895.65331234 673861.24480478 727902.94917506]
nuclei_within_radius = [159055]
nuclei_within_radius_distance = [140.90148267]

There were 1 nuclei found within the radius of 20000 nm
winning_nuclei = 159055
winning_nuclei_distance = 140.90148266678236

 For Bounding Box Search:
inside_nuclei = [159055]


At End: using return_id_0_if_no_matches = True
winning_nuclei = 159055
winning_nuclei_distance = 140.90148266678236
n_nuclei_in_radius = 1
n_nuclei_in_bbox = 1
winning_nucleus_id = 159055


# Exploring the Synapse Table

In [93]:
du.m65mat.NucleusSynapse()

ver materialization version,synapse_id synapse index within the segmentation,presyn segment_id of the presynaptic cell. Equivalent to Allen 'pt_root_id'.,postsyn segment_id of the postsynaptic cell. Equivalent to Allen 'pt_root_id'.,"synapse_x x coordinate of synapse centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_y y coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_z z coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.",synapse_size (EM voxels) scaled by (4x4x40)
30.00,10957,864691135087399350,864691135087399350,55164,103758,21202,20484
30.00,11021,864691135087399350,864691135087399350,55242,102912,21202,4780
30.00,11047,864691135087399350,864691135087399350,55336,102653,21202,3368
30.00,11065,864691135087399350,864691135087399350,55397,103990,21199,628
30.00,11075,864691135087399350,864691135087399350,55429,103423,21201,6292
30.00,11081,864691135087399350,864691135087399350,55444,103664,21203,2932
30.00,11203,864691135087399350,864691135087399350,55786,104910,21208,14656
30.00,11220,864691135087399350,864691135087399350,55778,102596,21203,2840
30.00,11240,864691135087399350,864691135087399350,56040,104665,21199,16628
30.00,11338,864691135087399350,864691135087399350,56036,101908,21200,264


In [98]:
beginning_direct_connections = (du.m65mat.NucleusSynapse() & 
 minnie.Decomposition.proj(presyn="segment_id") & 
 minnie.Decomposition.proj(postsyn="segment_id") & 
 "presyn != postsyn")

beginning_direct_connections

ver materialization version,synapse_id synapse index within the segmentation,presyn segment_id of the presynaptic cell. Equivalent to Allen 'pt_root_id'.,postsyn segment_id of the postsynaptic cell. Equivalent to Allen 'pt_root_id'.,"synapse_x x coordinate of synapse centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_y y coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_z z coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.",synapse_size (EM voxels) scaled by (4x4x40)
30.00,515874,864691135593498923,864691136723541757,67724,172488,21439,3964
30.00,519451,864691135986920451,864691135393987573,67244,132814,21355,1288
30.00,520250,864691135617800847,864691136816114148,68898,132056,21426,1768
30.00,526715,864691135307077446,864691136056321496,68598,184442,19902,3256
30.00,526750,864691135359859698,864691135415460794,68620,186396,19886,448
30.00,535902,864691136723550461,864691135385316693,65906,151160,20306,2328
30.00,536477,864691136723550461,864691135122295079,66568,153744,20344,6052
30.00,536537,864691136723550461,864691135122295079,66628,153942,20356,3096
30.00,536764,864691136723550461,864691136740407644,66906,152780,20380,3036
30.00,536901,864691136723550461,864691136740407644,67038,152502,20359,5764


# Part C: Getting the Faces of the Original Mesh

In [ ]:
"""
Pseudocode: 
1) Get the final mesh build of the final neuron (already present in the mesh)
2) Map the final mesh to the original mesh 

"""

In [132]:
import trimesh_utils as tu

In [111]:
original_mesh = du.fetch_segment_id_mesh(filtered_neuron.segment_id)

'filtered_neuron'

In [136]:
original_mesh_faces = tu.original_mesh_faces_map(original_mesh,
                                                filtered_neuron.mesh,
                                                exact_match=True,
                                                original_mesh_kdtree=original_mesh_kdtree)
original_mesh_faces

array([375513, 375512, 375511, ..., 969164, 969163, 969179], dtype=uint32)

In [137]:
recovered_mesh = original_mesh.submesh([original_mesh_faces],append=True)
nviz.plot_objects(recovered_mesh)

# Part D: Getting the Synapse Information

In [178]:
(keys_to_write,
 synapse_stats,
 total_error_synapse_ids) = pru.synapse_filtering(filtered_neuron,
                split_index,
                segment_id=None,
                return_synapse_filter_info = True,
                return_synapse_center_data = True,
                return_error_synapse_ids = True,
                mapping_threshold = 500,
                proof_version = 0,
                  plot_synapses=True,
                verbose = True)

For presyn: # valid synapses = 159, # error synapses  = 146
For postsyn: # valid synapses = 257, # error synapses  = 272
Displaying the Synapse Classifications


In [176]:
print("hi")

hi


In [161]:
# data_to_write = []
# synapse_stats = {}
# synapse_center_coordinates = {}
# total_error_synapse_ids = []

# for synapse_type in ["presyn","postsyn"]:
    
#     #3) Fetch the synapses and scale the centers
#     table_for_type = beginning_direct_connections & f"{synapse_type}={segment_id}"
#     synapse_ids, centroid_xs, centroid_ys, centroid_zs = table_for_type.fetch("synapse_id","synapse_x","synapse_y","synapse_z")
#     synapse_centers = np.vstack([centroid_xs,centroid_ys,centroid_zs]).T
#     synapse_centers_scaled = synapse_centers* [4, 4, 40]
    
    
#     #4) Find the distance of synapses to mesh
#     dist,closest_face = neuron_kd.query(synapse_centers_scaled)
#     errored_synapses_idx = np.where(dist>mapping_threshold)[0]
#     valid_synapses_idx = np.delete(np.arange(len(synapse_centers_scaled)),errored_synapses_idx)

#     n_errored_synapses = len(errored_synapses_idx)
#     n_valid_synapses = len(valid_synapses_idx)
#     total_synapses = n_errored_synapses + n_valid_synapses

#     if verbose:
#         print(f"For {synapse_type}: # valid synapses = {n_valid_synapses}, # error synapses  = {n_errored_synapses}")
        
#     """
#     6) For synapses to keep create a list of dictionaries saving off:
#     synaps_id
#     type (presyn or postsyn)
#     segment_id
#     split_id
#     nucleus_id
#     """

#     errored_synapses = synapse_ids[errored_synapses_idx]
#     non_errored_synapses = synapse_ids[valid_synapses_idx]
    
#     if len(errored_synapses) > 0:
#         total_error_synapse_ids += errored_synapses

#     local_data_to_write = [dict(synapse_id=syn,
#                           synapse_type=synapse_type,
#                           proof_version = proof_version,
#                           segment_id = segment_id,
#                           split_index = split_index) for syn in non_errored_synapses]
    
    
#     #7) Save of the stats on how many synapses of that type you started with and how many you finished with
#     local_synapse_stats = {f"n_valid_syn_{synapse_type}":n_valid_synapses,
#                            f"n_errored_syn_{synapse_type}":n_errored_synapses,
#                           }
#     synapse_stats.update(local_synapse_stats)
    
#     #8) Save of synapse centers into valid and error groups
#     local_synapse_centers = {f"valid_syn_centers_{synapse_type}":synapse_centers_scaled[valid_synapses_idx],
#                            f"errored_syn_centers_{synapse_type}":synapse_centers_scaled[errored_synapses_idx],
#                           }
#     synapse_center_coordinates.update(local_synapse_centers)

For presyn: # valid synapses = 159, # error synapses  = 146
For postsyn: # valid synapses = 257, # error synapses  = 272


In [168]:
nviz.plot_valid_error_synapses(filtered_neuron,
                              synapse_center_coordinates)